I import pandas so I can read the CSVs into dataframes

In [1]:
import pandas as pd
import numpy as np

I have pandas convert my csv's into dataframes

In [2]:
invoice_test = pd.read_csv('invoice_test.csv', low_memory=False)
invoice_train = pd.read_csv('invoice_train.csv', low_memory=False)
client_test = pd.read_csv('client_test.csv', low_memory=False)
client_train = pd.read_csv('client_train.csv', low_memory=False)
sample_submission = pd.read_csv('SampleSubmission.csv', low_memory=False)

print(invoice_test.shape, invoice_train.shape, client_test.shape, client_train.shape)

(887059, 16) (864499, 16) (58069, 5) (135493, 6)


In [3]:
client_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   disrict        135493 non-null  int64  
 1   client_id      135493 non-null  object 
 2   client_catg    135493 non-null  int64  
 3   region         135493 non-null  int64  
 4   creation_date  135493 non-null  object 
 5   target         135493 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 6.2+ MB


In [4]:
def feature_change(cl, inv):

    cl['client_catg'] = cl['client_catg'].astype('category')
    cl['disrict'] = cl['disrict'].astype('category')
    cl['region'] = cl['region'].astype('category')
    cl['region_group'] = cl['region'].apply(lambda x: 100 if x<100 else 300 if x>300 else 200)
    cl['creation_date'] = pd.to_datetime(cl['creation_date'])
    
    cl['coop_time'] = (2019 - cl['creation_date'].dt.year)*12 - cl['creation_date'].dt.month

    inv['counter_type'] = inv['counter_type'].map({"ELEC":1,"GAZ":0})
    inv['counter_statue'] = inv['counter_statue'].map({0:0,1:1,2:2,3:3,4:4,5:5,769:5,'0':0,'5':5,'1':1,'4':4,'A':0,618:5,269375:5,46:5,420:5})
    
    inv['invoice_date'] = pd.to_datetime(inv['invoice_date'], dayfirst=True)
    inv['invoice_month'] = inv['invoice_date'].dt.month
    inv['invoice_year'] = inv['invoice_date'].dt.year
    inv['is_weekday'] = ((pd.DatetimeIndex(inv.invoice_date).dayofweek) // 5 == 1).astype(float)
    inv['delta_index'] = inv['new_index'] - inv['old_index']
    
    return cl, inv

In [5]:
client_train1, invoice_train1 = feature_change(client_train, invoice_train)
client_test1, invoice_test1 = feature_change(client_test, invoice_test)

C:\Users\Nick_\AppData\Local\Temp\ipykernel_161172\2795683553.py:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  cl['creation_date'] = pd.to_datetime(cl['creation_date'])
C:\Users\Nick_\AppData\Local\Temp\ipykernel_161172\2795683553.py:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  cl['creation_date'] = pd.to_datetime(cl['creation_date'])


In [6]:
def agg_feature(invoice, client_df, agg_stat):
    
    invoice['delta_time'] = invoice.sort_values(['client_id','invoice_date']).groupby('client_id')['invoice_date'].diff().dt.days.reset_index(drop=True)
    agg_trans = invoice.groupby('client_id')[agg_stat+['delta_time']].agg(['mean','std','min','max'])
    
    agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = invoice.groupby('client_id').size().reset_index(name='transactions_count')
    agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')
    
    weekday_avg = invoice.groupby('client_id')[['is_weekday']].agg(['mean'])
    weekday_avg.columns = ['_'.join(col).strip() for col in weekday_avg.columns.values]
    weekday_avg.reset_index(inplace=True)
    client_df = pd.merge(client_df, weekday_avg, on='client_id', how='left')
    
    full_df = pd.merge(client_df, agg_trans, on='client_id', how='left')
    
    full_df['invoice_per_cooperation'] = full_df['transactions_count'] / full_df['coop_time']
    
    return full_df

In [7]:
aggs = {}
aggs['consommation_level_1'] = ['mean']
aggs['consommation_level_2'] = ['mean']
aggs['consommation_level_3'] = ['mean']
aggs['consommation_level_4'] = ['mean']
agg_trans = invoice_train.groupby(['client_id', 'counter_type']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

agg_trans1 = agg_trans

aggs = {}
aggs['consommation_level_1_mean'] = ['mean']
aggs['consommation_level_2_mean'] = ['mean']
aggs['consommation_level_3_mean'] = ['mean']
aggs['consommation_level_4_mean'] = ['mean']
agg_trans = agg_trans1.groupby(['client_id']).agg(aggs)
agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
agg_trans.reset_index(inplace=True)

df = (invoice_train.groupby('client_id')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))
agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

In [8]:
agg_stat_columns = [
 'tarif_type',
 'counter_number',
 'counter_statue',
 'counter_code',
 'reading_remarque',
 'consommation_level_1',
 'consommation_level_2',
 'consommation_level_3',
 'consommation_level_4',
 'old_index',
 'new_index',
 'months_number',
 'counter_type',
 'invoice_month',
 'invoice_year',
 'delta_index'
]

train_df1 = agg_feature(invoice_train1, client_train1, agg_stat_columns)
test_df1 = agg_feature(invoice_test1, client_test1, agg_stat_columns)

In [9]:
train_df1 = pd.merge(train_df1,agg_trans, on='client_id', how='left')

test_df1 = pd.merge(test_df1,agg_trans, on='client_id', how='left')

print(train_df1.columns)

Index(['disrict', 'client_id', 'client_catg', 'region', 'creation_date',
       'target', 'region_group', 'coop_time', 'is_weekday_mean',
       'transactions_count', 'tarif_type_mean', 'tarif_type_std',
       'tarif_type_min', 'tarif_type_max', 'counter_number_mean',
       'counter_number_std', 'counter_number_min', 'counter_number_max',
       'counter_statue_mean', 'counter_statue_std', 'counter_statue_min',
       'counter_statue_max', 'counter_code_mean', 'counter_code_std',
       'counter_code_min', 'counter_code_max', 'reading_remarque_mean',
       'reading_remarque_std', 'reading_remarque_min', 'reading_remarque_max',
       'consommation_level_1_mean', 'consommation_level_1_std',
       'consommation_level_1_min', 'consommation_level_1_max',
       'consommation_level_2_mean', 'consommation_level_2_std',
       'consommation_level_2_min', 'consommation_level_2_max',
       'consommation_level_3_mean', 'consommation_level_3_std',
       'consommation_level_3_min', 'consomma

In [10]:
def new_features(df):
    
    for col in agg_stat_columns:
        df[col+'_range'] = df[col+'_max'] - df[col+'_min']
        df[col+'_max_mean'] = df[col+'_max']/df[col+'_mean']
    
    return df

In [11]:
train_df2 = new_features(train_df1)
test_df2 = new_features(test_df1)

In [12]:
def drop(df):

    col_drop = ['client_id', 'creation_date']
    for col in col_drop:
        df.drop([col], axis=1, inplace=True)
    return df

In [13]:
train_df = drop(train_df2)
test_df = drop(test_df2)

In [14]:
y = train_df['target']
X = train_df.drop('target',axis=1)

feature_name = X.columns.tolist()

In [15]:
drop_col=['reading_remarque_max','counter_statue_min','counter_type_min','counter_type_max','counter_type_range',
          'tarif_type_max', 'delta_index_min', 'consommation_level_4_mean']

X = X.drop(drop_col, axis=1)
test_df = test_df.drop(drop_col, axis=1)

In [16]:
y

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
135488    0.0
135489    0.0
135490    0.0
135491    0.0
135492    0.0
Name: target, Length: 135493, dtype: float64

In [17]:
X

,disrict,client_catg,region,region_group,coop_time,is_weekday_mean,transactions_count,tarif_type_mean,tarif_type_std,tarif_type_min,...,new_index_max_mean,months_number_range,months_number_max_mean,counter_type_max_mean,invoice_month_range,invoice_month_max_mean,invoice_year_range,invoice_year_max_mean,delta_index_range,delta_index_max_mean
0,60,11,101,200,288,0.028571,35.0,11.0,0.0,11.0,...,1.741318,10.0,2.592593,1.0,11.0,1.794872,14.0,1.003907,1348.0,3.818482
1,69,11,107,200,199,0.054054,37.0,11.0,0.0,11.0,...,1.680649,6.0,1.850000,1.0,9.0,1.947368,14.0,1.003560,1017.0,2.164865
2,62,11,301,300,393,0.055556,18.0,11.0,0.0,11.0,...,1.323484,8.0,1.862069,1.0,9.0,1.663866,14.0,1.004006,2894.0,3.684399
3,69,11,105,200,265,0.050000,20.0,11.0,0.0,11.0,...,1.195595,2.0,1.428571,1.0,9.0,1.801802,7.0,1.001693,15.0,12.500000
4,62,11,303,300,50,0.285714,14.0,11.0,0.0,11.0,...,1.754937,2.0,1.076923,1.0,10.0,1.584906,4.0,1.001062,2258.0,2.581714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135488,62,11,304,300,173,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135489,63,11,311,300,74,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135490,63,11,311,300,85,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135491,60,11,101,200,300,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)

In [19]:
from flaml import AutoML
automl = AutoML()

In [20]:
settings = {
    "time_budget": 600,
    "metric": 'f1',  
    "task": 'classification',
    "log_file_name": 'zindi_experiment.log',
    "seed": 123456,
}

In [ ]:
automl.fit(X_train=X, y_train=y, **settings)

In [22]:
'''retrieve best config and best learner'''
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 9, 'num_leaves': 578, 'min_child_samples': 3, 'learning_rate': 1.0, 'log_max_bin': 10, 'colsample_bytree': 0.7670907141761506, 'reg_alpha': 0.0009765625, 'reg_lambda': 0.028390585536496545, 'FLAML_sample_size': 121943}
Best accuracy on validation data: 0.1139
Training duration of best run: 1.862 s


In [23]:
automl.model.estimator

LGBMClassifier(colsample_bytree=0.7670907141761506, learning_rate=1.0,
               max_bin=1023, min_child_samples=3, n_estimators=9,
               num_leaves=578, reg_alpha=0.0009765625,
               reg_lambda=0.028390585536496545, verbose=-1)

In [24]:
'''pickle and save the automl object'''
import pickle
with open('automl.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)
'''load pickled automl object'''
with open('automl.pkl', 'rb') as f:
    automl = pickle.load(f)

In [25]:
'''compute predictions of training dataset''' 
y_pred = automl.predict(X)
print('Predicted labels', y_pred)
print('True labels', y)
y_pred_proba = automl.predict_proba(X)[:,1]

Predicted labels [0. 0. 0. ... 0. 0. 0.]
True labels 0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
135488    0.0
135489    0.0
135490    0.0
135491    0.0
135492    0.0
Name: target, Length: 135493, dtype: float64


In [26]:
''' compute different metric values on training dataset'''
from flaml.ml import sklearn_metric_loss_score
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred, y))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y))

accuracy = 0.9539016775774394
roc_auc = 0.8513459904039535
log_loss = 0.1764543146443834


In [28]:
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [31]:
'''compute predictions of testing dataset''' 
y_pred = automl.predict(test_df)
print('Predicted labels', y_pred)

Predicted labels [0. 1. 0. ... 0. 0. 0.]


In [32]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58069 entries, 0 to 58068
Columns: 105 entries, disrict to delta_index_max_mean
dtypes: category(3), float64(100), int64(2)
memory usage: 45.8 MB


In [33]:
sample_submission["client_id"]

0           test_Client_0
1           test_Client_1
2          test_Client_10
3         test_Client_100
4        test_Client_1000
               ...       
58064    test_Client_9995
58065    test_Client_9996
58066    test_Client_9997
58067    test_Client_9998
58068    test_Client_9999
Name: client_id, Length: 58069, dtype: object

In [34]:
len(y_pred)

58069

In [35]:
prediction_array = pd.DataFrame(y_pred, columns=["label"])

In [37]:
submission = pd.DataFrame({
        "client_id": sample_submission["client_id"],
        "target": prediction_array["label"]
    })
submission.to_csv('submission.csv', index=False)

In [38]:
from flaml.data import get_output_from_log
time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = \
    get_output_from_log(filename=settings['log_file_name'], time_budget=240)
for config in config_history:
    print(config)

{'Current Learner': 'lgbm', 'Current Sample': 10000, 'Current Hyper-parameters': {'n_estimators': 4, 'num_leaves': 4, 'min_child_samples': 20, 'learning_rate': 0.09999999999999995, 'log_max_bin': 8, 'colsample_bytree': 1.0, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.0, 'FLAML_sample_size': 10000}, 'Best Learner': 'lgbm', 'Best Hyper-parameters': {'n_estimators': 4, 'num_leaves': 4, 'min_child_samples': 20, 'learning_rate': 0.09999999999999995, 'log_max_bin': 8, 'colsample_bytree': 1.0, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.0, 'FLAML_sample_size': 10000}}
{'Current Learner': 'xgboost', 'Current Sample': 10000, 'Current Hyper-parameters': {'n_estimators': 4, 'max_leaves': 4, 'min_child_weight': 0.9999999999999993, 'learning_rate': 0.09999999999999995, 'subsample': 1.0, 'colsample_bylevel': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 0.0009765625, 'reg_lambda': 1.0, 'FLAML_sample_size': 10000}, 'Best Learner': 'xgboost', 'Best Hyper-parameters': {'n_estimators': 4, 'max_leaves': 4, 'm

In [39]:
import matplotlib.pyplot as plt
import numpy as np

plt.title('Learning Curve')
plt.xlabel('Wall Clock Time (s)')
plt.ylabel('Validation Accuracy')
plt.scatter(time_history, 1 - np.array(valid_loss_history))
plt.step(time_history, 1 - np.array(best_valid_loss_history), where='post')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'